In [1]:
# Import torch & Check CUDA availability
import torch

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

True
1
0


In [2]:
# Get CUDA device name
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

NVIDIA A100 80GB PCIe


#### Import Reddit

In [3]:
from torch_geometric.datasets import Reddit
import torch_geometric.transforms as T

# Import dataset from PyTorch Geometric
dataset = Reddit(root="/dfs6/pub/seminl1/Reddit", transform=T.ToSparseTensor())
data = dataset[0]
data = data.pin_memory()

# Print information about the dataset
print(f'Dataset: {dataset}')
print('-------------------')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of nodes: {data.x.shape[0]}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

# Print information about the graph
print(f'\nGraph:')
print('------')
print(f'Edges are directed: {data.is_directed()}')
print(f'Graph has isolated nodes: {data.has_isolated_nodes()}')
print(f'Graph has loops: {data.has_self_loops()}')

Dataset: Reddit()
-------------------
Number of graphs: 1
Number of nodes: 232965
Number of features: 602
Number of classes: 41

Graph:
------
Edges are directed: False
Graph has isolated nodes: False
Graph has loops: False


#### Graph Information

In [4]:
# Print first element
print(f'Graph: {data}')

Graph: Data(x=[232965, 602], y=[232965], train_mask=[232965], val_mask=[232965], test_mask=[232965], adj_t=[232965, 232965, nnz=114615892])


In [5]:
# Node feature matrix information
print(f'x = {data.x.shape}')
print(data.x)

x = torch.Size([232965, 602])
tensor([[ 1.2334,  9.0430, -0.9233,  ..., -0.2579,  0.3112, -0.3772],
        [-0.1386, -0.2022,  0.1277,  ...,  0.1563,  0.1048, -0.6534],
        [-0.1330, -0.1962, -0.0296,  ...,  0.0358,  0.2864,  0.2744],
        ...,
        [-0.0614, -0.2022,  0.9698,  ...,  1.1064, -1.4323, -0.2398],
        [-0.1606, -0.2022, -0.0892,  ...,  0.7440, -0.5046, -2.2288],
        [ 0.0929,  0.2822,  0.1768,  ...,  0.2196,  0.5967,  0.5588]])


In [6]:
# Adjacency matrix for the edges
print(data.adj_t)

SparseTensor(row=tensor([     0,      0,      0,  ..., 232964, 232964, 232964]),
             col=tensor([   242,    249,    524,  ..., 231806, 232594, 232634]),
             size=(232965, 232965), nnz=114615892, density=0.21%)


In [7]:
# Density of adjacency matrix
print(data.adj_t.density()*100)

0.21118520090487738


In [8]:
# Ground-truth labels
print(f'y = {data.y.shape}')
print(data.y)

y = torch.Size([232965])
tensor([30, 17, 18,  ...,  3, 13, 13])


In [9]:
# Train mask
print(f'train_mask = {data.train_mask.shape}')
print(data.train_mask)

train_mask = torch.Size([232965])
tensor([False,  True, False,  ...,  True,  True, False])


#### Single-layer GCN

In [10]:
# Create a simple GCN with only one GCN layer
import torch.nn.functional as F

from torch.nn import Linear
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.gcn1 = GCNConv(dataset.num_features, dataset.num_classes)
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.02)

    def forward(self, x, adj_t):
        x = self.gcn1(x, adj_t)
        z = F.log_softmax(x, dim=1)
        return x, z

#### To get GFLOPS result

In [11]:
def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

def train(model, data):
    """Train a GNN model and return the trained model."""
    criterion = torch.nn.CrossEntropyLoss().cuda()
    optimizer = model.optimizer
    epochs = 100

    model.train()
    for epoch in range(epochs):
        # Training
        optimizer.zero_grad()
        h, out = model(data.x, data.adj_t)
        loss = criterion(out, data.y)
        #acc = accuracy(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        # Print metrics every 10 epochs
        #if(epoch % 10 == 0):
            #print('Epoch:', epoch)
            #print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: {acc*100:>6.2f}%')
          
    return model, h, out

##### #CPU_Cores = 32

In [11]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

# Create GCN model
gcn = GCN()
print(gcn)
print()

# Train
start.record()
gcn_model, gcn_output, final_output = train(gcn.to('cuda:0'), data.to('cuda:0', non_blocking=True))
end.record()
torch.cuda.synchronize()
elapsed_time = start.elapsed_time(end)
print('Elapsed Time (100 Epochs):', elapsed_time*0.001, 'seconds')

GCN(
  (gcn1): GCNConv(602, 41)
)

Elapsed Time (100 Epochs): 22.55403125 seconds


##### #CPU_Core = 16

In [11]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

# Create GCN model
gcn = GCN()
print(gcn)
print()

# Train
start.record()
gcn_model, gcn_output, final_output = train(gcn.to('cuda:0'), data.to('cuda:0', non_blocking=True))
end.record()
torch.cuda.synchronize()
elapsed_time = start.elapsed_time(end)
print('Elapsed Time (100 Epochs):', elapsed_time*0.001, 'seconds')

GCN(
  (gcn1): GCNConv(602, 41)
)

Elapsed Time (100 Epochs): 22.53441796875 seconds


##### #CPU_Core = 8

In [11]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

# Create GCN model
gcn = GCN()
print(gcn)
print()

# Train
start.record()
gcn_model, gcn_output, final_output = train(gcn.to('cuda:0'), data.to('cuda:0', non_blocking=True))
end.record()
torch.cuda.synchronize()
elapsed_time = start.elapsed_time(end)
print('Elapsed Time (100 Epochs):', elapsed_time*0.001, 'seconds')

GCN(
  (gcn1): GCNConv(602, 41)
)

Elapsed Time (100 Epochs): 22.51006640625 seconds


#### To get accuracy result

In [11]:
def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

def train(model, data):
    """Train a GNN model and return the trained model."""
    criterion = torch.nn.CrossEntropyLoss().cuda()
    optimizer = model.optimizer
    epochs = 100

    model.train()
    for epoch in range(epochs+1):
        # Training
        optimizer.zero_grad()
        h, out = model(data.x, data.adj_t)
        loss = criterion(out, data.y)
        acc = accuracy(out.argmax(dim=1), data.y)
        loss.backward()
        optimizer.step()

        # Print metrics every 10 epochs
        if(epoch % 10 == 0):
            print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: {acc*100:>6.2f}%')
          
    return model, h, out

In [12]:
# Create GCN model
gcn = GCN()
print(gcn)
print()

# Train
gcn_model, gcn_output, final_output = train(gcn.to('cuda:0'), data.to('cuda:0', non_blocking=True))

GCN(
  (gcn1): GCNConv(602, 41)
)

Epoch   0 | Train Loss: 4.114 | Train Acc:   0.76%
Epoch  10 | Train Loss: 0.561 | Train Acc:  90.50%
Epoch  20 | Train Loss: 0.408 | Train Acc:  92.26%
Epoch  30 | Train Loss: 0.355 | Train Acc:  93.07%
Epoch  40 | Train Loss: 0.327 | Train Acc:  93.50%
Epoch  50 | Train Loss: 0.307 | Train Acc:  93.76%
Epoch  60 | Train Loss: 0.293 | Train Acc:  93.97%
Epoch  70 | Train Loss: 0.282 | Train Acc:  94.12%
Epoch  80 | Train Loss: 0.273 | Train Acc:  94.25%
Epoch  90 | Train Loss: 0.265 | Train Acc:  94.35%
Epoch 100 | Train Loss: 0.259 | Train Acc:  94.44%
